1) Generate dim_date (2020–2025)

In [1]:
import pandas as pd
dates=[]
for y in range(2020, 2026):
    for m in range(1,13):
        dates.append({"date_id":y*100+m,"year":y,"month":m})
dim_date = pd.DataFrame(dates)
dim_date.to_csv("data_warehouse1/dim_date.csv",index=False)


**2) Process `passenger2020–2025` → produce two tables (`fact_route_monthly.csv` & `fact_passenger_monthly.csv`)**


In [2]:
import pandas as pd, numpy as np, glob, os

# --------------------------------------------------
# 1. Read and merge passenger T-100 data
# --------------------------------------------------
files = glob.glob("passenger2020-2025/*.csv")
dfs = []
for f in files:
    df = pd.read_csv(f, low_memory=False)
    df.columns = [c.lower().strip() for c in df.columns]
    keep = [c for c in [
        "year","month","carrier","origin","origin_airport_id",
        "dest","dest_airport_id","passengers","seats","distance"
    ] if c in df.columns]
    df = df[keep]
    dfs.append(df)

raw = pd.concat(dfs, ignore_index=True)

# Numeric cleaning
for c in ["year","month","seats","origin_airport_id","dest_airport_id"]:
    if c in raw:
        raw[c] = pd.to_numeric(raw[c], errors="coerce").astype("Int64")

for c in ["passengers","distance"]:
    if c in raw:
        raw[c] = pd.to_numeric(raw[c], errors="coerce")

raw["origin"] = raw["origin"].str.upper().str.strip()
raw["dest"]   = raw["dest"].str.upper().str.strip()

raw = raw.dropna(subset=["year","month","origin","dest"])

# ⭐⭐⭐ Only study AA ⭐⭐⭐
raw = raw[raw["carrier"] == "AA"]


# --------------------------------------------------
# 2. fact_route_monthly (O-D × month)
# --------------------------------------------------
agg = {"passengers": "sum"}
if "seats" in raw:     agg["seats"] = "sum"
if "distance" in raw:  agg["distance"] = "median"

fr = raw.groupby(["year","month","origin","dest"], as_index=False).agg(agg)
fr["date_id"] = fr["year"] * 100 + fr["month"]

# ⭐ No longer using dim_airport → export IATA directly ⭐
fr = fr[["year","month","date_id","origin","dest","passengers"]
        + (["seats"] if "seats" in fr else [])
        + (["distance"] if "distance" in fr else [])]


# --------------------------------------------------
# 3. fact_passenger_monthly (airport × month, in + out)
# --------------------------------------------------
outb = fr.groupby(["date_id","year","month","origin"], as_index=False)["passengers"].sum() \
         .rename(columns={"origin":"airport","passengers":"pax_out"})

inb = fr.groupby(["date_id","year","month","dest"], as_index=False)["passengers"].sum() \
         .rename(columns={"dest":"airport","passengers":"pax_in"})

# Outer join to merge
fp = outb.merge(inb, on=["date_id","year","month","airport"], how="outer").fillna(0)
fp["pax"] = fp["pax_out"] + fp["pax_in"]

fp = fp[["year","month","date_id","airport","pax_out","pax_in","pax"]]


# --------------------------------------------------
# 4. Save (only fact tables, no dim tables)
# --------------------------------------------------
os.makedirs("data_warehouse1", exist_ok=True)

fr.to_csv("data_warehouse1/fact_route_monthly.csv", index=False)
fp.to_csv("data_warehouse1/fact_passenger_monthly.csv", index=False)

print("✓ fact_route_monthly.csv generated")
print("✓ fact_passenger_monthly.csv generated")
print("(No dim_airport — processed exactly according to your requirement)")


✓ fact_route_monthly.csv generated
✓ fact_passenger_monthly.csv generated
(No dim_airport — processed exactly according to your requirement)


**3. Process `airport_state.txt` (airport–state mapping) → `dim_airport_state.csv`**


In [3]:
import re
import pandas as pd

records = []

# Matching pattern: ", AL:" and "(ANB)"
pattern = re.compile(r",\s*([A-Z]{2}):.*\(([A-Z0-9]{2,4})\)$")

with open("airport_state.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        m = pattern.search(line)
        if m:
            state_abbr = m.group(1)   # AL / AK / CA ...
            iata = m.group(2)         # ANB / LAX / ANC ...
            records.append((iata, state_abbr))

df = pd.DataFrame(records, columns=["airport", "state_abbr"])
df = df.drop_duplicates().sort_values("airport").reset_index(drop=True)

df.to_csv("data_warehouse1/dim_airport_state.csv", index=False)
print("✓ Success! Number of parsed airports:", len(df))
print(df.head())


✓ Success! Number of parsed airports: 1484
  airport state_abbr
0     03A         AK
1     04A         AK
2     05A         AK
3     06A         AK
4     1AK         AK


**4) Process delay causes (delaycause2020–2025) → `fact_delay_monthly.csv`**


In [4]:
import pandas as pd, glob

files = glob.glob("delaycause2020.1-2025.7/*.csv")

all_columns = []

for f in files:
    d = pd.read_csv(f, nrows=5)   # Read only the first few rows to improve speed
    cols = tuple(d.columns.str.lower().str.strip())  # Normalize column names
    all_columns.append((f, cols))

# --- Check unique sets of column names ---
unique_structures = {}

for f, cols in all_columns:
    unique_structures.setdefault(cols, []).append(f)

# Output results
print("\n=== Column Structure Check Results ===")
for cols, files_with_these_cols in unique_structures.items():
    print("\nColumns:", cols)
    print("Files:", files_with_these_cols)



=== Column Structure Check Results ===

Columns: ('year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay')
Files: ['delaycause2020.1-2025.7\\Airline_Delay_Cause.csv']


In [5]:
import pandas as pd
import os

# --------------------------------------------------
# Part 1: Read delaycause & clean (your original code)
# --------------------------------------------------

# Read CSV
fd = pd.read_csv("delaycause2020.1-2025.7/Airline_Delay_Cause.csv")

# ⭐⭐⭐ Only study AA (the most important line) ⭐⭐⭐
fd = fd[fd["carrier"] == "AA"]

# Convert column names to lowercase
fd.columns = [c.lower().strip() for c in fd.columns]

# Convert to numeric
delay_cols = [
    "arr_delay", "carrier_delay", "weather_delay",
    "nas_delay", "security_delay", "late_aircraft_delay"
]

for c in delay_cols:
    fd[c] = pd.to_numeric(fd[c], errors="coerce")

# Create date_id
fd["date_id"] = fd["year"].astype(int) * 100 + fd["month"].astype(int)

# Airport code
fd["airport"] = fd["airport"].str.upper().str.strip()

drop_cols = ["carrier_name", "airport_name"]
fd = fd.drop(columns=[c for c in drop_cols if c in fd.columns])

# Create directory
os.makedirs("data_warehouse1", exist_ok=True)

# Save fact_delay_monthly
fd.to_csv("data_warehouse1/fact_delay_monthly.csv", index=False)
print("✓ fact_delay_monthly generated!")


✓ fact_delay_monthly generated!


**5) Process `stormevent2020–2025` → `fact_weather_state_monthly.csv` & `fact_weather_state_event_type_monthly.csv`**


In [6]:
import pandas as pd, glob

files = glob.glob("stormevent2020-2025/*.csv")

all_columns = []

for f in files:
    d = pd.read_csv(f, nrows=5)   # Read only the first few rows to improve speed
    cols = tuple(d.columns.str.lower().str.strip())  # Normalize column names
    all_columns.append((f, cols))

# --- Check unique column name combinations ---
unique_structures = {}

for f, cols in all_columns:
    unique_structures.setdefault(cols, []).append(f)

# Output results
print("\n=== Column Structure Check Results ===")
for cols, files_with_these_cols in unique_structures.items():
    print("\nColumns:", cols)
    print("Files:", files_with_these_cols)



=== Column Structure Check Results ===

Columns: ('begin_yearmonth', 'begin_day', 'begin_time', 'end_yearmonth', 'end_day', 'end_time', 'episode_id', 'event_id', 'state', 'state_fips', 'year', 'month_name', 'event_type', 'cz_type', 'cz_fips', 'cz_name', 'wfo', 'begin_date_time', 'cz_timezone', 'end_date_time', 'injuries_direct', 'injuries_indirect', 'deaths_direct', 'deaths_indirect', 'damage_property', 'damage_crops', 'source', 'magnitude', 'magnitude_type', 'flood_cause', 'category', 'tor_f_scale', 'tor_length', 'tor_width', 'tor_other_wfo', 'tor_other_cz_state', 'tor_other_cz_fips', 'tor_other_cz_name', 'begin_range', 'begin_azimuth', 'begin_location', 'end_range', 'end_azimuth', 'end_location', 'begin_lat', 'begin_lon', 'end_lat', 'end_lon', 'episode_narrative', 'event_narrative', 'data_source')
Files: ['stormevent2020-2025\\StormEvents_details-ftp_v1.0_d2020_c20250702.csv', 'stormevent2020-2025\\StormEvents_details-ftp_v1.0_d2021_c20250520.csv', 'stormevent2020-2025\\StormEvents_

In [7]:
import pandas as pd, numpy as np, glob, re, os

# ---------- 1. State name → state abbreviation mapping ----------

state_map = {
    # 50 States
    "ALABAMA": "AL", "ALASKA": "AK", "ARIZONA": "AZ", "ARKANSAS": "AR",
    "CALIFORNIA": "CA", "COLORADO": "CO", "CONNECTICUT": "CT",
    "DELAWARE": "DE", "DISTRICT OF COL*": "DC",
    "FLORIDA": "FL", "GEORGIA": "GA", "HAWAII": "HI", "IDAHO": "ID",
    "ILLINOIS": "IL", "INDIANA": "IN", "IOWA": "IA", "KANSAS": "KS",
    "KENTUCKY": "KY", "LOUISIANA": "LA", "MAINE": "ME", "MARYLAND": "MD",
    "MASSACHUSETTS": "MA", "MICHIGAN": "MI", "MINNESOTA": "MN",
    "MISSISSIPPI": "MS", "MISSOURI": "MO", "MONTANA": "MT",
    "NEBRASKA": "NE", "NEVADA": "NV", "NEW HAMPSHIRE": "NH",
    "NEW JERSEY": "NJ", "NEW MEXICO": "NM", "NEW YORK": "NY",
    "NORTH CAROLINA": "NC", "NORTH DAKOTA": "ND", "OHIO": "OH",
    "OKLAHOMA": "OK", "OREGON": "OR", "PENNSYLVANIA": "PA",
    "RHODE ISLAND": "RI", "SOUTH CAROLINA": "SC", "SOUTH DAKOTA": "SD",
    "TENNESSEE": "TN", "TEXAS": "TX", "UTAH": "UT", "VERMONT": "VT",
    "VIRGINIA": "VA", "WASHINGTON": "WA", "WEST VIRGINIA": "WV",
    "WISCONSIN": "WI", "WYOMING": "WY",

    # U.S. Territories used by NOAA StormEvents
    "PUERTO RICO": "PR",
    "GUAM": "GU",
    "VIRGIN ISLANDS": "VI",
    "U.S. VIRGIN ISLANDS": "VI",
    "AMERICAN SAMOA": "AS",
    "NORTHERN MARIANA ISLANDS": "MP",
    "MARSHALL ISLANDS": "MH",
    "MICRONESIA": "FM",
    "PALAU": "PW",

    # Special NOAA categories
    "ATLANTIC NORTH": "AN",
    "ATLANTIC SOUTH": "ASO",
    "GULF OF ALASKA": "GOA",
    "LAKE HURON": "LH",
    "LAKE MICHIGAN": "LM",
    "LAKE SUPERIOR": "LS",
    "LAKE ERIE": "LE",
    "LAKE ONTARIO": "LO",
    "PACIFIC NORTH": "PN",
    "PACIFIC SOUTH": "PS",
    "HAWAII WATERS": "HW",
    "CARIBBEAN": "CB"
}

files = glob.glob("stormevent2020-2025/*.csv")
dfs = []

for f in files:
    df = pd.read_csv(f, low_memory=False)
    df.columns = [c.lower().strip() for c in df.columns]

    # ---------- 2. Time ----------
    df["begin_ts"] = pd.to_datetime(df["begin_date_time"], errors="coerce")
    df["year"] = df["begin_ts"].dt.year
    df["month"] = df["begin_ts"].dt.month
    df["date_id"] = df["year"] * 100 + df["month"]

    # ---------- 3. Standardize state names ----------
    df["state_full"] = df["state"].str.upper().str.strip()
    df["state_abbr"] = df["state_full"].map(state_map)

    # Drop records whose states cannot be mapped (e.g., overseas territories)
    df = df.dropna(subset=["state_abbr"])

    # ---------- 4. Convert damage amount ----------
    def dmg(x):
        if pd.isna(x):
            return 0.0
        s = str(x).strip().upper()
        m = re.match(r"([\d\.]+)\s*([KMB])?", s)
        if not m:
            return 0.0
        val = float(m.group(1))
        unit = m.group(2) or ""
        return val * {"K": 1e3, "M": 1e6, "B": 1e9}.get(unit, 1)

    df["damage"] = df["damage_property"].apply(dmg)

    # ---------- 5. Severe event flag ----------
    severe_set = {
        "THUNDERSTORM WIND","HAIL","BLIZZARD","HEAVY RAIN","FLASH FLOOD",
        "FLOOD","TORNADO","HIGH WIND","WINTER STORM","ICE STORM",
        "HURRICANE","TROPICAL STORM","WILDFIRE"
    }
    df["event_type_clean"] = df["event_type"].str.upper().str.strip()
    df["severe_flag"] = df["event_type_clean"].isin(severe_set).astype(int)

    dfs.append(df[["state_abbr", "date_id", "event_id",
                   "event_type_clean", "severe_flag", "damage"]])

wx = pd.concat(dfs, ignore_index=True)

# ---------- 6. Aggregate by state + month ----------
fact_weather_state = wx.groupby(
    ["state_abbr", "date_id"], as_index=False
).agg(
    events=("event_id", "count"),
    severe_events=("severe_flag", "sum"),
    total_damage=("damage", "sum")
)

# ---------- 7. Normalized weather index ----------
max_severe = fact_weather_state["severe_events"].max()
max_damage = fact_weather_state["total_damage"].max()

fact_weather_state["wx_index"] = (
    (fact_weather_state["severe_events"] / (max_severe if max_severe > 0 else 1)) * 60 +
    (np.log1p(fact_weather_state["total_damage"]) /
     (np.log1p(max_damage) if max_damage > 0 else 1)) * 40
)

os.makedirs("data_warehouse1", exist_ok=True)
fact_weather_state.to_csv("data_warehouse1/fact_weather_state_monthly.csv",
                          index=False)

print("✓ fact_weather_state_monthly.csv completed")

# ---------- New: Aggregate by state + month + event type ----------
fact_weather_state_type = wx.groupby(
    ["state_abbr", "date_id", "event_type_clean"],
    as_index=False
).agg(
    event_count=("event_id", "count")
)

# Export
fact_weather_state_type.to_csv(
    "data_warehouse1/fact_weather_state_event_type_monthly.csv",
    index=False
)

print("✓ fact_weather_state_event_type_monthly.csv completed")


C:\Users\14277\AppData\Local\Temp\ipykernel_31560\2274165273.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["begin_ts"] = pd.to_datetime(df["begin_date_time"], errors="coerce")
C:\Users\14277\AppData\Local\Temp\ipykernel_31560\2274165273.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["begin_ts"] = pd.to_datetime(df["begin_date_time"], errors="coerce")
C:\Users\14277\AppData\Local\Temp\ipykernel_31560\2274165273.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["begin_ts"] = pd.to_datetime(df["begin_date_time"], errors="coerce")
C:\Users\14277\AppData

✓ fact_weather_state_monthly.csv completed
✓ fact_weather_state_event_type_monthly.csv completed


**6) Process Gas/Fuel Prices → `fact_fuel_price_monthly.csv`**


In [8]:
import pandas as pd

df_raw = pd.read_excel(
    r"./gas price2020-2025/EER_EPJK_PF4_RGC_DPGm.xls",
    sheet_name="Data 1"
)

print(df_raw.head(20))
print("Columns:", df_raw.columns.tolist())




       Back to Contents  \
0             Sourcekey   
1                  Date   
2   1990-04-15 00:00:00   
3   1990-05-15 00:00:00   
4   1990-06-15 00:00:00   
5   1990-07-15 00:00:00   
6   1990-08-15 00:00:00   
7   1990-09-15 00:00:00   
8   1990-10-15 00:00:00   
9   1990-11-15 00:00:00   
10  1990-12-15 00:00:00   
11  1991-01-15 00:00:00   
12  1991-02-15 00:00:00   
13  1991-03-15 00:00:00   
14  1991-04-15 00:00:00   
15  1991-05-15 00:00:00   
16  1991-06-15 00:00:00   
17  1991-07-15 00:00:00   
18  1991-08-15 00:00:00   
19  1991-09-15 00:00:00   

   Data 1: U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)  
0                                EER_EPJK_PF4_RGC_DPG                                  
1   U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Pr...                                  
2                                                0.54                                  
3                                               0.515                           

In [9]:
import pandas as pd
import numpy as np
import os

file = r"./gas price2020-2025/EER_EPJK_PF4_RGC_DPGm.xls"

# Read the "Data 1" sheet
df = pd.read_excel(file, sheet_name="Data 1")

# Standardize column names
df.columns = ["date_raw", "price_raw"]

# Drop the first two metadata rows (Sourcekey, Description)
df = df.iloc[2:].reset_index(drop=True)

# Parse datetime
df["date"] = pd.to_datetime(df["date_raw"], errors="coerce")

# Drop empty rows
df = df.dropna(subset=["date"])

# Extract year and month
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["date_id"] = df["year"] * 100 + df["month"]

# ⭐⭐⭐ Keep only data from 202001 and later ⭐⭐⭐
df = df[df["date_id"] >= 202001]

# Parse price
df["price"] = pd.to_numeric(df["price_raw"], errors="coerce")

# Drop rows without price
df = df.dropna(subset=["price"])

# Ensure monthly average (the dataset is already monthly)
fact_fuel = df.groupby("date_id", as_index=False)["price"].mean()

# Export to data warehouse
os.makedirs("data_warehouse1", exist_ok=True)
fact_fuel.to_csv("data_warehouse1/fact_fuel_price_monthly.csv", index=False)

print("✓ fact_fuel_price_monthly.csv generated")


✓ fact_fuel_price_monthly.csv generated


7.generate

In [10]:
import pandas as pd

# --------------------------------------------------
# 1. Read the two fact files you provided
# --------------------------------------------------

fact_state = pd.read_csv("data_warehouse1/fact_weather_state_monthly.csv")
fact_evt = pd.read_csv("data_warehouse1/fact_weather_state_event_type_monthly.csv")


# --------------------------------------------------
# 2. Create dim_event_type (dim_weather_type)
# --------------------------------------------------

dim_event_type = (
    pd.DataFrame(
        sorted(fact_evt["event_type_clean"].dropna().unique()),
        columns=["event_type_clean"]
    )
    .reset_index()
    .rename(columns={"index": "event_type_id"})
)

# Surrogate key starts from 1
dim_event_type["event_type_id"] = dim_event_type["event_type_id"] + 1

# Save the file
dim_event_type.to_csv("data_warehouse1/dim_event_type.csv", index=False)

print("\n✓ dim_event_type.csv generated!")
print(dim_event_type.head())



✓ dim_event_type.csv generated!
   event_type_id       event_type_clean
0              1  ASTRONOMICAL LOW TIDE
1              2              AVALANCHE
2              3               BLIZZARD
3              4          COASTAL FLOOD
4              5        COLD/WIND CHILL


In [11]:
# --------------------------------------------------
# Create dim_state — extracted from dim_airport_state
# --------------------------------------------------

aps = pd.read_csv("data_warehouse1/dim_airport_state.csv")

dim_state = (
    pd.DataFrame(
        sorted(aps["state_abbr"].dropna().unique()),
        columns=["state_abbr"]
    )
    .reset_index()
    .rename(columns={"index": "state_id"})
)

# Surrogate key starts from 1
dim_state["state_id"] = dim_state["state_id"] + 1

# Save file
dim_state.to_csv("data_warehouse1/dim_state.csv", index=False)

print("✓ dim_state.csv generated based on dim_airport_state")
print(dim_state.head())


✓ dim_state.csv generated based on dim_airport_state
   state_id state_abbr
0         1         AK
1         2         AL
2         3         AR
3         4         AZ
4         5         CA
